### Data Loader

In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA, NMF
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.manifold import TSNE
from sklearn.mixture import GaussianMixture
from sklearn.mixture import BayesianGaussianMixture
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import LabelEncoder
import seaborn as sns

In [2]:
csf_data = pd.read_csv("merged_csf_cross_sectional_call_rate_pau_data_0205.xls", low_memory=False)
plasma_data = pd.read_csv("merged_plasma_cross_sectional_call_rate_data_0130.xls", low_memory=False)

In [3]:
csf_data['Status_at_draw_mapping'].value_counts()
# csf_data['Status_at_draw'].value_counts()

Status_at_draw_mapping
AD     744
PD     738
CO     665
FTD     46
DLB     37
Name: count, dtype: int64

In [4]:
csf_significant_rows = pd.read_csv("csf_significant_rows_0205.csv")
plasma_significant_rows = pd.read_csv("plasma_significant_rows_0203.csv")

### Fill in the missing Values

In [48]:
data = csf_data
print(data.shape)

(4677, 3667)


In [49]:
x_columns = [col for col in csf_data.columns if col.startswith('X')]
len(x_columns)

3622

In [50]:
protein_list = list(csf_significant_rows['Analytes'])
print(len(protein_list))
# protein_list

3020


In [51]:
data['Status_at_draw_mapping'].value_counts()

Status_at_draw_mapping
AD     744
PD     738
CO     665
FTD     46
DLB     37
Name: count, dtype: int64

In [52]:
statuses_to_keep = ['CO', 'AD', 'PD', 'FTD', 'DLB']
# statuses_to_keep = ['CO', 'AD', 'PD', 'Prodromal', 'ADAD']

filtered_data = data[data['Status_at_draw_mapping'].isin(statuses_to_keep)]
# filtered_data = data[data['Status_at_draw'].isin(statuses_to_keep)]

filtered_data.shape

(2230, 3667)

In [53]:
filtered_data = filtered_data.rename(columns={'final_decision': 'PET_imaging'})

columns_to_keep = ['UniquePhenoID', 'DrawDate', 'Project_x', 'Project_y', 'Age_at_draw', 'Sex', 'AT_class', 'PET_imaging', 'T1_pTau217',
       'T2_pTau181', 'Status_at_draw_mapping', 'Status_at_draw', 'Final_Status']
print(columns_to_keep)

selected_protein_columns = [col for col in data.columns if col in protein_list]

selected_columns = list(columns_to_keep) + selected_protein_columns

selected_data = filtered_data[selected_columns]

selected_data.shape

['UniquePhenoID', 'DrawDate', 'Project_x', 'Project_y', 'Age_at_draw', 'Sex', 'AT_class', 'PET_imaging', 'T1_pTau217', 'T2_pTau181', 'Status_at_draw_mapping', 'Status_at_draw', 'Final_Status']


(2230, 3033)

In [54]:
# Check which columns contain non-float values
non_float_columns = selected_data.iloc[:,13:].applymap(lambda x: isinstance(x, (float, int))).all() == False
non_float_columns_indices = non_float_columns[non_float_columns].index

if not non_float_columns_indices.empty:
    print(f"Columns with non-float values: {list(non_float_columns_indices)}")
else:
    print("All columns are float type.")

C:\Users\x.ying1\AppData\Local\Temp\ipykernel_31680\3170815002.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  non_float_columns = selected_data.iloc[:,13:].applymap(lambda x: isinstance(x, (float, int))).all() == False


All columns are float type.


In [55]:
# Check for NA values in selected_data
na_counts = selected_data.iloc[:,13:].isna().sum()

# Get columns with NA values
na_columns = na_counts[na_counts > 0]

# Print the total number of NA values and columns with NA values
total_na = na_counts.sum()
print(f"Total number of NA values in selected_data: {total_na}")
if not na_columns.empty:
    print("Columns with NA values and their counts:")
    print(na_columns)
else:
    print("No NA values in selected_data.")

Total number of NA values in selected_data: 128392
Columns with NA values and their counts:
X10521.10      13
X11368.32      20
X11425.31      32
X8535.102      27
X5648.28       24
             ... 
X12348.46       7
X12551.3      102
X9898.161      37
X8845.2        30
X12904.180     33
Length: 3015, dtype: int64


In [56]:
import numpy as np

np.random.seed(42)

def bootstrap_impute(series):
    observed = series.dropna()
    n_missing = series.isna().sum()
    if n_missing > 0 and len(observed) > 0:
        imputed_values = np.random.choice(observed, size=n_missing, replace=True)
        series.loc[series.isna()] = imputed_values
    return series

num_cols = selected_data.columns[13:]

selected_data[num_cols] = (
    selected_data.groupby("Status_at_draw", group_keys=False)[num_cols]
    .apply(lambda g: g.apply(bootstrap_impute, axis=0))
)

selected_data[num_cols] = selected_data[num_cols].fillna(selected_data[num_cols].median())

na_counts_after = selected_data[num_cols].isna().sum()
total_na_after = na_counts_after.sum()
print(f"Total number of NA values in selected_data after filling: {total_na_after}")

if not na_counts_after[na_counts_after > 0].empty:
    print("Columns with remaining NA values and their counts:")
    print(na_counts_after[na_counts_after > 0])
else:
    print("No NA values in selected_data after filling.")

C:\Users\x.ying1\AppData\Local\Temp\ipykernel_31680\2335237837.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data[num_cols] = (
C:\Users\x.ying1\AppData\Local\Temp\ipykernel_31680\2335237837.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data[num_cols] = selected_data[num_cols].fillna(selected_data[num_cols].median())


Total number of NA values in selected_data after filling: 0
No NA values in selected_data after filling.


### Logistic Regression

#### ptau217

In [57]:
selected_data['Age_at_draw'].value_counts()

Age_at_draw
72.0    84
71.0    83
73.0    83
74.0    78
70.0    73
        ..
44.1     1
71.5     1
51.7     1
48.1     1
57.6     1
Name: count, Length: 321, dtype: int64

In [58]:
selected_data['Sex'].value_counts()

Sex
Male      1089
Female    1054
F           50
M           37
Name: count, dtype: int64

In [59]:
selected_data['T1_pTau217'].value_counts()

T1_pTau217
T-    248
T+    152
Name: count, dtype: int64

In [71]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, f1_score, balanced_accuracy_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

# Set random seed for reproducibility
np.random.seed(42)

# Step 1: Filter the dataset to include only 'CO' and 'AD' samples
filtered_data_new = selected_data[selected_data['Status_at_draw'].isin(['CO', 'AD'])]
# filtered_data_new = selected_data[selected_data['Status_at_draw'].isin(['CO', 'PD'])]

# Show the distribution of 'Status_at_draw' after filtering
print("Status_at_draw Value Counts:\n", filtered_data_new['Status_at_draw'].value_counts())

# Step 2: Drop samples with missing values in the required columns
filtered_data_new = filtered_data_new.dropna(subset=['Age_at_draw', 'Sex', 'T1_pTau217'])

# Step 3: Convert 'Sex' from 'F'/'M' to 'Female'/'Male'
filtered_data_new['Sex'] = filtered_data_new['Sex'].replace({
    'F': 'Female',
    'M': 'Male'
})

# filtered_data_new['Status_at_draw'] = filtered_data_new['Status_at_draw'].map({'CO': 0, 'AD': 1})
# filtered_data_new['Status_at_draw'] = filtered_data_new['Status_at_draw'].map({'CO': 1, 'AD': 0})

# Step 4: Encode 'Sex' into numerical values: 0 = Female, 1 = Male
filtered_data_new['Sex'] = filtered_data_new['Sex'].map({'Female': 0, 'Male': 1})

# Step 5: Encode 'T1_pTau217' into numerical values: 0 = T-, 1 = T+
filtered_data_new['T1_pTau217'] = filtered_data_new['T1_pTau217'].map({'T-': 0, 'T+': 1})

# Step 6: Check distributions after encoding
print("Sex Value Counts:\n", filtered_data_new['Sex'].value_counts())
print("T1_pTau217 Value Counts:\n", filtered_data_new['T1_pTau217'].value_counts())

# Step 7: Define features and label
X = filtered_data_new[['Age_at_draw', 'Sex', 'T1_pTau217']]
print(X.head())
y = filtered_data_new['Status_at_draw']

# Drop rows with missing feature values (if any)
X = X.dropna()
y = y.loc[X.index]  # Align y with the filtered X

# Step 8: Encode categorical labels to numerical format
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
# y_encoded = y

# Step 9: Split the dataset into training and testing sets (stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.3, random_state=42, stratify=y_encoded
)

# Step 10: Train logistic regression model
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Display the mapping from encoded labels back to original class names
class_labels = label_encoder.inverse_transform(np.unique(y_encoded))
print("Class Labels:", class_labels)

# Step 11: Make predictions and compute probability scores
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probability for class 1 (usually AD)

# Step 12: Evaluate performance
accuracy = accuracy_score(y_test, y_pred)
balanced_acc = balanced_accuracy_score(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average="macro")
auc_score = roc_auc_score(y_test, y_pred_proba)

# Step 13: Print performance metrics
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Balanced Accuracy: {balanced_acc * 100:.2f}%")
print(f"Macro F1-score: {f1_macro:.4f}")
print("Classification Report:\n", classification_report(y_test, y_pred, digits=4))
print(f"ROC AUC Score: {auc_score:.4f}")


Status_at_draw Value Counts:
 Status_at_draw
AD    744
CO    665
Name: count, dtype: int64
Sex Value Counts:
 Sex
0    223
1    164
Name: count, dtype: int64
T1_pTau217 Value Counts:
 T1_pTau217
0    239
1    148
Name: count, dtype: int64
    Age_at_draw  Sex  T1_pTau217
83         73.0    1           0
84         68.0    1           0
85         70.0    0           0
86         66.0    0           0
88         70.0    0           0
Class Labels: ['AD' 'CO']
Accuracy: 84.62%
Balanced Accuracy: 74.20%
Macro F1-score: 0.7565
Classification Report:
               precision    recall  f1-score   support

           0     0.6667    0.5600    0.6087        25
           1     0.8854    0.9239    0.9043        92

    accuracy                         0.8462       117
   macro avg     0.7760    0.7420    0.7565       117
weighted avg     0.8387    0.8462    0.8411       117

ROC AUC Score: 0.9096


In [66]:
X_final = pd.concat([X_train, X_test])
y_final = pd.Series(np.concatenate([y_train, y_test]), index=X_final.index)
y_final_labels = label_encoder.inverse_transform(y_final)

X_final['Status_at_draw'] = y_final_labels

print("\n[Final Sample Distributions Used in Model]")

# Status_at_draw distribution
print("\nStatus_at_draw Distribution:")
print(X_final['Status_at_draw'].value_counts())

# Sex distribution (0 = Female, 1 = Male)
print("\nSex Distribution:")
print(X_final['Sex'].map({0: 'Female', 1: 'Male'}).value_counts())

# T1_pTau217 distribution (0 = T-, 1 = T+)
print("\nT1_pTau217 Distribution:")
print(X_final['T1_pTau217'].map({0: 'T-', 1: 'T+'}).value_counts())


[Final Sample Distributions Used in Model]

Status_at_draw Distribution:
Status_at_draw
CO     304
FTD      2
Name: count, dtype: int64

Sex Distribution:
Sex
Female    182
Male      124
Name: count, dtype: int64

T1_pTau217 Distribution:
T1_pTau217
T-    238
T+     68
Name: count, dtype: int64


In [67]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_test, y_pred)

tn, fp, fn, tp = conf_matrix.ravel()

sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0  # True Positive Rate / Recall
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0  # True Negative Rate

print("\n[Final Performance Metrics]")
print(f"Accuracy     : {accuracy * 100:.1f}%")
print(f"AUC          : {auc_score:.3f}")
print(f"Specificity  : {specificity:.3f}")
print(f"Sensitivity  : {sensitivity:.3f}")



[Final Performance Metrics]
Accuracy     : 98.9%
AUC          : 0.495
Specificity  : 1.000
Sensitivity  : 0.000


In [68]:
tn, fp, fn, tp

(91, 0, 1, 0)

In [69]:
conf_matrix

array([[91,  0],
       [ 1,  0]], dtype=int64)

#### Amyloid PET Imaging

In [39]:
filtered_data['PET_imaging'].value_counts()

PET_imaging
negative    677
positive    233
Name: count, dtype: int64

In [77]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, accuracy_score, roc_auc_score,
    f1_score, balanced_accuracy_score, confusion_matrix
)
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

# Set random seed for reproducibility
np.random.seed(42)

# Step 1: Filter the dataset to include only 'CO' and 'AD' samples
filtered_data_new = selected_data[selected_data['Status_at_draw'].isin(['CO', 'AD'])]
# filtered_data_new = selected_data[selected_data['Status_at_draw'].isin(['CO', 'PD'])]

# Show the distribution of 'Status_at_draw' after filtering
print("Status_at_draw Value Counts:\n", filtered_data_new['Status_at_draw'].value_counts())

# Step 2: Drop samples with missing values in the required columns
filtered_data_new = filtered_data_new.dropna(subset=['Age_at_draw', 'Sex', 'PET_imaging'])

# Step 3: Convert 'Sex' from 'F'/'M' to 'Female'/'Male'
filtered_data_new['Sex'] = filtered_data_new['Sex'].replace({
    'F': 'Female',
    'M': 'Male'
})

# Step 4: Encode 'Sex' into numerical values: 0 = Female, 1 = Male
filtered_data_new['Sex'] = filtered_data_new['Sex'].map({'Female': 0, 'Male': 1})

# Step 5: Encode 'PET_imaging' into numerical values: 0 = negative, 1 = positive
filtered_data_new['PET_imaging'] = filtered_data_new['PET_imaging'].map({'negative': 0, 'positive': 1})

# Step 6: Check distributions after encoding
print("Sex Value Counts:\n", filtered_data_new['Sex'].value_counts())
print("PET_imaging Value Counts:\n", filtered_data_new['PET_imaging'].value_counts())

# Step 7: Define features and label
X = filtered_data_new[['Age_at_draw', 'Sex', 'PET_imaging']]
y = filtered_data_new['Status_at_draw']

# Drop rows with missing feature values (if any)
X = X.dropna()
y = y.loc[X.index]

# Step 8: Encode categorical labels to numerical format
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Step 9: Split the dataset into training and testing sets (stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.3, random_state=42, stratify=y_encoded
)

# Step 10: Train logistic regression model
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Display the class label mapping
class_labels = label_encoder.inverse_transform(np.unique(y_encoded))
print("Class Labels:", class_labels)

# Step 11: Predict and compute probabilities
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probability for AD class

# Step 12: Evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
balanced_acc = balanced_accuracy_score(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average="macro")
auc_score = roc_auc_score(y_test, y_pred_proba)

# Step 13: Specificity & Sensitivity
conf_matrix = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = conf_matrix.ravel()
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0

# Final Output
print("\n[Final Performance Metrics]")
print(f"Accuracy     : {accuracy * 100:.2f}%")
print(f"Balanced Accuracy: {balanced_acc * 100:.2f}%")
print(f"AUC          : {auc_score:.3f}")
print(f"Specificity  : {specificity:.3f}")
print(f"Sensitivity  : {sensitivity:.3f}")
print("Classification Report:\n", classification_report(y_test, y_pred, digits=4))

Status_at_draw Value Counts:
 Status_at_draw
AD    744
CO    665
Name: count, dtype: int64
Sex Value Counts:
 Sex
0    189
1    139
Name: count, dtype: int64
PET_imaging Value Counts:
 PET_imaging
0    279
1     49
Name: count, dtype: int64
Class Labels: ['AD' 'CO']

[Final Performance Metrics]
Accuracy     : 96.97%
Balanced Accuracy: 94.68%
AUC          : 0.989
Specificity  : 0.917
Sensitivity  : 0.977
Classification Report:
               precision    recall  f1-score   support

           0     0.8462    0.9167    0.8800        12
           1     0.9884    0.9770    0.9827        87

    accuracy                         0.9697        99
   macro avg     0.9173    0.9468    0.9313        99
weighted avg     0.9711    0.9697    0.9702        99



In [75]:
X_final = pd.concat([X_train, X_test])
y_final = pd.Series(np.concatenate([y_train, y_test]), index=X_final.index)
y_final_labels = label_encoder.inverse_transform(y_final)

X_final['Status_at_draw'] = y_final_labels

print("\n[Final Sample Distributions Used in Model]")

# Status_at_draw distribution
print("\nStatus_at_draw Distribution:")
print(X_final['Status_at_draw'].value_counts())

# Sex distribution (0 = Female, 1 = Male)
print("\nSex Distribution:")
print(X_final['Sex'].map({0: 'Female', 1: 'Male'}).value_counts())

print("\nPET_imaging Distribution:")
print(X_final['PET_imaging'].map({0: 'negative', 1: 'positive'}).value_counts())


[Final Sample Distributions Used in Model]

Status_at_draw Distribution:
Status_at_draw
CO     287
FTD      2
Name: count, dtype: int64

Sex Distribution:
Sex
Female    167
Male      122
Name: count, dtype: int64

PET_imaging Distribution:
PET_imaging
negative    277
positive     12
Name: count, dtype: int64
